Загрузка библиотек и данных

In [1]:
!pip install simpletransformers==0.61.13
!pip uninstall transformers
!pip install transformers==4.10.0
!pip install wikipedia
!pip install googledrivedownloader

     |████████████████████████████████| 221 kB 5.2 MB/s 
     |████████████████████████████████| 124 kB 40.0 MB/s 
     |████████████████████████████████| 287 kB 32.3 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 1.7 MB 35.2 MB/s 
     |████████████████████████████████| 2.9 MB 37.0 MB/s 
     |████████████████████████████████| 8.3 MB 46.8 MB/s 
     |████████████████████████████████| 3.3 MB 37.3 MB/s 
     |████████████████████████████████| 1.2 MB 28.8 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 38.9 MB/s 
     |████████████████████████████████| 139 kB 48.9 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 180 kB 45.0 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 125 kB 28.5 MB/s 
     |█████████████████

Found existing installation: transformers 4.11.3
Uninstalling transformers-4.11.3:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.7/dist-packages/transformers-4.11.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/transformers/*
Proceed (y/n)? y
  Successfully uninstalled transformers-4.11.3
     |████████████████████████████████| 2.8 MB 5.3 MB/s 
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=9e380a541c716b5644e74a00d25cad44f234df32eea72fbdae68f9fd1d3c3b7c
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('wordnet')
stopwords = stopwords.words("english")
lemmatizer = nltk.stem.WordNetLemmatizer() 

from nltk.tokenize import RegexpTokenizer, word_tokenize, sent_tokenize
main_tokenizer = RegexpTokenizer(r'\w+',)
sec_tokenizer = RegexpTokenizer(r'\S+')
 
from sklearn.externals import joblib
import numpy as np
from google.colab import output
import urllib
import difflib
import wikipedia
import pandas as pd
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='13Nuwm7BV-4RXI9JqjPTDE9rcdupkKqlF',
                                    dest_path='/Data/AIIJC/aiijc_1578_goodFromTrain_pretrained.model')
gdd.download_file_from_google_drive(file_id='1Gl_eWLhS-svO6nqn5OqtIbHPfZDn1pM0',
                                    dest_path='/Data/AIIJC/newTrain.csv')

Функции для предобработки текста

In [4]:
def normal_form(word): #Получение нормальной формы слова
  word = word.lower()
  return word
 
def clean_html(html): #Очистка html
  soup = BeautifulSoup(BeautifulSoup(html, "lxml").text)
  return str(soup.body)

def get_good_tokens(text): #Выделение ключевых слов
  good_tokens = []

  for tokens in tokenizer(text)[1]:
    for token in tokens:
      token = normal_form(token)
      if token not in stopwords:
        good_tokens.append(token)
  return good_tokens

def tokenizer(text): #Токенизация текста в обработанные и необработанные токены
  raw_tokens = sec_tokenizer.tokenize(text)
  clean_tokens = main_tokenizer.tokenize_sents(raw_tokens)
  
  nClean_tokens = []
  for i in range(len(clean_tokens)):
    nClean_tokens.append([])
    for m in range(len(clean_tokens[i])):
      if normal_form(clean_tokens[i][m]) != 's':
        nClean_tokens[i].append(normal_form(clean_tokens[i][m]))

  return (raw_tokens, nClean_tokens)
 
def similarity(s1, s2): #Нахождение коэффициента схожести между двумя строками
  normalized1 = s1.lower()
  normalized2 = s2.lower()
  matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
  return matcher.ratio()

def part_extractor(data,question,step,part_length,corrAns=''): #Функция выделения релевантного фрагмента (Текст, вопрос, длинна фрагмента)
  good_tokens = get_good_tokens(question)
  
  tokens = tokenizer(data)
  
  isIntext = corrAns.lower() in data.lower()
    
  for i in range(step-(len(tokens[0]) % step)): #Увеличение количества токенов до кратного длины части
    tokens[0].append('')
    tokens[1].append('')
 

  match_counter = 0 #Счетчик точных совпадений токенов
  best_part = '' #Лучшая часть
  max_match_qty = 0 #Максимальное количество совпавших токенов
 
  main_clrTokens = tokens[1]
  main_tokens = tokens[0]
  
  for i in range(0,len(tokens[0])-1,part_length): #Нахождение наиболее релевантной части текста
    tokens = main_tokens[i:i+part_length-1]
    clrTokens = main_clrTokens[i:i+part_length-1]

    for good_token in good_tokens:
      if in_tokens(good_token,clrTokens):
        match_counter += 1

    if match_counter > max_match_qty:
      max_match_qty = match_counter
      best_part = tokens
      
   
    match_counter = 0
  
  

  fin = '' #Восстановление текста
  for i in best_part:
    fin += (i+' ')

  
  return fin,isIntext

def in_tokens(token,text):
  for i in text:
    for m in i:
      if token == m:
        return True
  return False

Загрузка данных и модели

In [5]:
train = pd.read_csv("/Data/AIIJC/newTrain.csv",index_col=0, error_bad_lines=False,sep=",")
train.predict_answer = ''
train

,question,answer,link_1,link_2,predict_answer
id,,,,,
1,What is the name of the locality that was name...,Antiene,https://en.wikipedia.org/wiki/List_of_cities_a...,https://en.wikipedia.org/wiki/Wikipedia:Naming...,
2,Who was the first woman to be selected as a Ti...,Mary Sullivan,https://en.wikipedia.org/wiki/Sylvia_Earle,https://en.wikipedia.org/wiki/Isabella_Bird,
3,What 15th century friar was a notable writer a...,Kanutus Johannis,https://en.wikipedia.org/wiki/The_Canterbury_T...,https://en.wikipedia.org/wiki/Scriptorium,
4,What does the Unstrut river form?,"the border between the Canton of St. Gallen, t...",https://en.wikipedia.org/wiki/Unstrut,https://en.wikipedia.org/wiki/Category:Unstrut...,
5,Who was the eighteenth Shah of Shirvan?,Afridun the Martyr,https://en.wikipedia.org/wiki/Ahmad_of_Shirvan,https://en.wikipedia.org/wiki/Afridun_I,
...,...,...,...,...,...
3996,What is the street named after?,Alexandre-Antonin Taché,https://en.wikipedia.org/wiki/List_of_streets_...,https://en.wikipedia.org/wiki/Street_or_road_name,
3997,Who was the student of Joseph Beuys?,Anatol Herzfeld,https://en.wikipedia.org/wiki/Joseph_Beuys,https://en.wikipedia.org/wiki/Kunstakademie_D%...,
3998,What is the name of the technology that has be...,Village Tronic,https://en.wikipedia.org/wiki/Multi-monitor,https://en.wikipedia.org/wiki/Videotelephony,


In [6]:
model = joblib.load('/Data/AIIJC/aiijc_1578_goodFromTrain_pretrained.model')

model.args.max_seq_length = 512
model.args.silent = True

Оценка модели на первой 1000 вопросов

In [ ]:
allCounter = 0 #Счетчик всех вопросов
corrCounter = 0 #Счетчик идеальных ответов
simCounter = 0 #Счетчик схожести ответов
errorCounter = 0 #Счетчик ошибок парсера
intext1Counter = 0 #Счетчики нахождения ответа в тексте
intext2Counter = 0 
inPart1_Counter = 0 #Счетчики нахождения ответа в выделенных фрагментах
inPart2_Counter = 0 
 
for i in range(0,1000): #Проверка модели на первой тысяче вопросов тренировочной выборки
  question = train.question.iloc[i]
  
  good_tokens = get_good_tokens(question) #Получаем ключевые слова вопроса

  #Загрузка статей википедии
  try:
    link_1 = train.link_1.iloc[i].replace('https://en.wikipedia.org/wiki/','') #Убераем начало ссылки
    link_1 = urllib.parse.unquote(link_1) #Заменяем кривые символы на оригинал
    data_1 = wikipedia.page(link_1,auto_suggest=False).content #Парсим страничку вики
    data_1 = data_1.replace('\n',' ')
  except:
    errorCounter += 1
  try:
    link_2 = train.link_2.iloc[i].replace('https://en.wikipedia.org/wiki/','') #Убераем начало ссылки
    link_2 = urllib.parse.unquote(link_2) #Заменяем кривые символы на оригинал
    data_2 = wikipedia.page(link_2,auto_suggest=False).content #Парсим страничку вики
    data_2 = data_2.replace('\n',' ')
  except:
    errorCounter += 1

  #Выделение фрагмента и предсказание по двум статьям
  try:
    res = part_extractor(data_1,question,16,64,train.answer.iloc[i])
    context = res[0]
    if res[1]:
      intext1Counter += 1
    if train.answer.iloc[i].lower() in res[0].lower():
      inPart1_Counter += 1
  except:
    errorCounter += 1
 
  try:
    res = part_extractor(data_2,question,16,32,train.answer.iloc[i])
    context += ' ' + res[0]
    if res[1]:
      intext2Counter += 1
    if train.answer.iloc[i].lower() in res[0].lower():
      inPart2_Counter += 1
  except:
    errorCounter += 1


  predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]

  #Если ответ не был найден, перебираем прочие длины фрагмента
  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_1,question,16,64)[0]
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass

  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_2,question,16,64)[0]
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass

  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_1,question,16,128)[0]
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass
  
  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_2,question,16,128)[0]
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass
 
  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_1,question,16,256)[0]
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass
 
  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_2,question,16,256)[0]
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass

  #Статистика
  allCounter += 1

  if str(predict[0]['answer'][0]).lower() == str(train.answer.iloc[i]).lower():
    corrCounter += 1
    print('Верный ответ - ' + str(train.answer.iloc[i]))
 
  if (i % 5) == 0:
    output.clear()
 
  simCounter += similarity(str(predict[0]['answer'][0]).lower(),str(train.answer.iloc[i]).lower())
 
  print('Количество правильных ответов - ' + str(corrCounter) + '/' + str(allCounter))
 
  train.predict_answer.iloc[i] = predict[0]['answer'][0]

Количество правильных ответов - 385/996
Верный ответ - Dublin
Количество правильных ответов - 386/997
Количество правильных ответов - 386/998
Количество правильных ответов - 386/999
Количество правильных ответов - 386/1000


In [ ]:
print('Количество идеальных ответов: ' + str(corrCounter))
print('Количество ошибок парсинга: ' + str(errorCounter))

Количество идеальных ответов: 386
Количество ошибок парсинга: 26


In [ ]:
print('Средний процент схожести ответов: ' + str(simCounter/i))
print('Вероятность правильного ответа: '+ str(corrCounter/allCounter))
print('Процент нахождения правильного ответа в первой статье: ' + str(intext1Counter/allCounter))
print('Процент нахождения правильного ответа во второй статье: ' + str(intext2Counter/allCounter))
print('Процент нахождения ответа в первом фрагменте: ' + str(inPart1_Counter/allCounter))
print('Процент нахождения ответа во втором фрагменте: ' + str(inPart2_Counter/allCounter))
print('Вероятность успешно выделить фрагмент из первой статьи(Если в ней есть ответ): ' + str(inPart1_Counter/intext1Counter))
print('Вероятность успешно выделить фрагмент из второй статьи(Если в ней есть ответ): ' + str(inPart2_Counter/intext2Counter))
print('Вероятность успешно ответить при верно выделенном фрагменте: ' + str(corrCounter/((inPart1_Counter+inPart2_Counter*0.2))))

Средний процент схожести ответов: 0.5700366564061863
Вероятность правильного ответа: 0.386
Процент нахождения правильного ответа в первой статье: 0.48
Процент нахождения правильного ответа во второй статье: 0.261
Процент нахождения ответа в первом фрагменте: 0.369
Процент нахождения ответа во втором фрагменте: 0.102
Вероятность успешно выделить фрагмент из первой статьи(Если в ней есть ответ): 0.76875
Вероятность успешно выделить фрагмент из второй статьи(Если в ней есть ответ): 0.39080459770114945
Вероятность успешно ответить при верно выделенном фрагменте: 0.9912686183872625
